## 如何在超参数和模型不变的情况下复现深度学习模型结果？

#### 在深度学习网络模型复现中，存在即使超参数和模型完全一样也不能完全复现模型结果的情况，主要原因是随机算法和随机数存在于模型各个部分的原因，而这将会导致一些问题，比如在优化模型阶段无法准确的去调整模型的超参数，因为使用相同的超参数每次得到的实验结果都不一样。这篇文章主要尝试分析和解决这个问题。
#### 从目前的了解来看，随机的情况主要是发生在两个方面，一个是随机数，主要产生于数据上，例如各种参数的随机初始化，二是随机算法，主要运用在torch框架内（本问主要针对torch），torch框架内有很多算法并不是‘deterministic’算法，torch官方对‘deterministic’算法的定义是：当输入一定时，经过算法计算之后的输出也是一定的。所以当一个算法是‘nondeterministic’时，输入一定的情况下，输出并不一定。
#### 所以为了解决这个问题，需要让数据和算法都是确定的。对于使用的随机数，往往是算法产生的伪随机数，是通过种子算出来的，所以当种子一定的时候，生成的随机数就是确定的了。对于随机算法，有些算法可以通过系统的设定变成确定的，而有些算法系统不支持设定成确定的，所以能做的是把这些能使用确定算法的算法设置好。
#### 下面分别说明数据和随机算法分别要具体设置哪些内容
#### 1、需要设定随机种子的对象：
#### 1）random.seed(seed) ——python的random库
#### 2）np.random.seed(seed)——numpy的random库
#### 3）torch.manual_seed(seed) ——torch中所有在cpu上初始化的随机数
#### 4）torch.cuda.manual_seed_all(seed)——torch的gpu上初始化的随机数
#### 2、需要进行设定的随机算法：
#### 1）torch.backends.cudnn.deterministic = True ——英伟达的cuDNN框架的一个属性设置，为True时设置这个框架中的所有算法是不是用确定性的算法。
#### 2）torch.use_deterministic_algorithms(mode,warn_only=False) ——torch中的所有算法是否使用确定性算法。

In [3]:
import torch
import numpy as np
import random

def setup_seed(seed):
     
     '''
     seed可以是任意的正整数
     '''
     
     #为torch的cpu设置随机种子
     torch.manual_seed(seed) 
     
     #为torch的gpu设置随机种子
     torch.cuda.manual_seed_all(seed)
     
     #为numpy设置随机种子   
     np.random.seed(seed)
    
     #为python的random库设置随机种子
     random.seed(seed)
    
     #torch使用的cuda框架是英伟达研发的，而英伟达使用的框架是cuDNN（CUDA Deep Neural Network library），这个属性控制基于这个框架的算法都是确定的
     torch.backends.cudnn.deterministic = True
    
     #torch中的一些方法是'deterministic',另外一些是'nondeterministic'的，使用下面这个函数可以让整个流程使用'deterministic'的方法
     #如果使用的方法是'nondeterministic'的，根据warn_only设置为False和True会分别进行报错和警告
     torch.use_deterministic_algorithms(True,warn_only=False)


In [5]:
#执行上面所有的设置
setup_seed(20)

#### 官方对于torch.use_deterministic_algorithms(mode,warn_only=False)的一些说明:

```python
#参数
mode (bool) – 如果设置为True, 默认情况下为nondeterministic的操作将转换成使用deterministic的算法，或者是抛出一个运行错误. 如果设置为False,则允许nondeterministic的操作.

#关键字参数
warn_only (bool, optional) – 如果设置为True, 没有deterministic的操作将抛出一个警告，而不是一个运行错误，默认设置为False。

#下面这些默认使用'nondeterministic'的函数或方法，当mode=True时，会强制使用'deterministic'的模型:

torch.nn.Conv1d when called on CUDA tensor

torch.nn.Conv2d when called on CUDA tensor

torch.nn.Conv3d when called on CUDA tensor

torch.nn.ConvTranspose1d when called on CUDA tensor

torch.nn.ConvTranspose2d when called on CUDA tensor

torch.nn.ConvTranspose3d when called on CUDA tensor

torch.bmm() when called on sparse-dense CUDA tensors

torch.Tensor.__getitem__() when attempting to differentiate a CPU tensor and the index is a list of tensors

torch.Tensor.index_put() with accumulate=False

torch.Tensor.index_put() with accumulate=True when called on a CPU tensor

torch.Tensor.put_() with accumulate=True when called on a CPU tensor

torch.Tensor.scatter_add_() when input dimension is one and called on a CUDA tensor

torch.gather() when input dimension is one and called on a CUDA tensor that requires grad

torch.index_add() when called on CUDA tensor

torch.index_select() when attempting to differentiate a CUDA tensor

torch.repeat_interleave() when attempting to differentiate a CUDA tensor

torch.Tensor.index_copy() when called on a CPU or CUDA tensor


#下面这些默认使用'nondeterministic'的函数或方法，当mode=True时，会抛出runTimeError:

torch.nn.AvgPool3d when attempting to differentiate a CUDA tensor

torch.nn.AdaptiveAvgPool2d when attempting to differentiate a CUDA tensor

torch.nn.AdaptiveAvgPool3d when attempting to differentiate a CUDA tensor

torch.nn.MaxPool3d when attempting to differentiate a CUDA tensor

torch.nn.AdaptiveMaxPool2d when attempting to differentiate a CUDA tensor

torch.nn.FractionalMaxPool2d when attempting to differentiate a CUDA tensor

torch.nn.FractionalMaxPool3d when attempting to differentiate a CUDA tensor

torch.nn.functional.interpolate() when attempting to differentiate a CUDA tensor and one of the following modes is used:

linear

bilinear

bicubic

trilinear

torch.nn.ReflectionPad1d when attempting to differentiate a CUDA tensor

torch.nn.ReflectionPad2d when attempting to differentiate a CUDA tensor

torch.nn.ReflectionPad3d when attempting to differentiate a CUDA tensor

torch.nn.ReplicationPad1d when attempting to differentiate a CUDA tensor

torch.nn.ReplicationPad2d when attempting to differentiate a CUDA tensor

torch.nn.ReplicationPad3d when attempting to differentiate a CUDA tensor

torch.nn.NLLLoss when called on a CUDA tensor

torch.nn.CTCLoss when attempting to differentiate a CUDA tensor

torch.nn.EmbeddingBag when attempting to differentiate a CUDA tensor when mode='max'

torch.Tensor.scatter_add_() when input dimension is larger than one and called on a CUDA tensor

torch.gather() when input dimension is larger than one and called on a CUDA tensor that requires grad

torch.Tensor.put_() when accumulate=False

torch.Tensor.put_() when accumulate=True and called on a CUDA tensor

torch.histc() when called on a CUDA tensor

torch.bincount() when called on a CUDA tensor

torch.kthvalue() with called on a CUDA tensor

torch.median() with indices output when called on a CUDA tensor

torch.nn.functional.grid_sample() when attempting to differentiate a CUDA tensor

#官方示例：
>>> torch.use_deterministic_algorithms(True)

# Forward mode nondeterministic error
>>> torch.randn(10).index_copy(0, torch.tensor([0]), torch.randn(1))
...
RuntimeError: index_copy does not have a deterministic implementation...

# Backward mode nondeterministic error
>>> torch.randn(10, requires_grad=True, device='cuda').index_select(0, torch.tensor([0], device='cuda')).backward()
...
RuntimeError: index_add_cuda_ does not have a deterministic implementation...

```